In [1]:
import pandas as pd
import numpy as np
import pprint

In [2]:
df1 = pd.read_csv("ML Lab 4 Data - Sheet1.csv")
df1 = df1.drop(['Day'], axis=1)
df1

,Outlook,Temperature,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [3]:
def find_entropy(df):
  Class = df.keys()[-1]
  values = df[Class].unique()
  entropy = 0
  for value in values:
    prob = df[Class].value_counts()[value]/len(df[Class])
    entropy += -prob * np.log2(prob)
  return np.float(entropy)

In [4]:
def find_entropy_attribute(df, attribute):
  Class = df.keys()[-1]
  target_values = df[Class].unique()
  attribute_values = df[attribute].unique()
  avg_entropy = 0
  for value in attribute_values:
    entropy = 0
    for value1 in target_values:
      num = len(df[attribute][df[attribute] == value][df[Class] == value1])
      den = len(df[attribute][df[attribute] == value])
      prob = num/den
      entropy += -prob * np.log2(prob + 0.000001)
    avg_entropy += (den/len(df))*entropy
  return np.float(avg_entropy)

In [5]:
def find_winner(df):
  IG = []
  for key in df.keys()[:-1]:
    IG.append(find_entropy(df) - find_entropy_attribute(df, key))
  return df.keys()[:-1][np.argmax(IG)]

In [6]:
def get_subtable(df, attribute, value):
  return df[df[attribute] == value].reset_index(drop = True)

In [7]:
def buildtree(df, tree = None):
  node = find_winner(df)
  attvalue = np.unique(df[node])
  Class = df.keys()[-1]
  if tree is None:
    tree = {}
    tree[node] = {}
  for value in attvalue:
    subtable = get_subtable(df,node,value)
    Clvalue, counts = np.unique(subtable[Class], return_counts = True)
    if len(counts) == 1:
      tree[node][value] = Clvalue[0]
    else:
      tree[node][value] = buildtree(subtable)
  return tree

In [8]:
tree = buildtree(df1)
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


C:\Users\Prash\AppData\Local\Temp\ipykernel_3564\2942177733.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(entropy)
C:\Users\Prash\AppData\Local\Temp\ipykernel_3564\3472808418.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(avg_entropy)
C:\Users\Prash\AppData\Local\Temp\ipykernel_3564\2942177733.p

In [9]:
def predict(inst, tree):
  for node in tree.keys():
    value = inst[node]
    tree = tree[node][value]
    prediction = 0
    if type(tree) is dict:
      prediction = predict(inst, tree)
    else:
      prediction = tree
  return prediction

In [10]:
Y_label = []
for i in range(len(df1)):
  inst = df1.iloc[i,:]
  prediction = predict(inst, tree)
  Y_label.append(prediction)
print(Y_label)

['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']


In [11]:
from sklearn import metrics
print(metrics.classification_report(df1.iloc[:,-1], Y_label))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         9

    accuracy                           1.00        14
   macro avg       1.00      1.00      1.00        14
weighted avg       1.00      1.00      1.00        14

